In [78]:
import numpy as np
import pandas as pd
import scipy as sp
import math

Генерируем ковариационную матрицу:(моделируем данные (см. Аллен 4.4))

In [79]:
d=6 #размерность наблюдений
n=72 #число наблюдений
alpha=0.05
generating_mean=np.zeros(d)
covar_generating_matrix=[[1,0.3966,0.3688,0.1764,-0.4632,0.2939],[0.3966,1,0.0232,-0.0854,0.0193,0.2191],[0.3688,0.0232,1,0.0494,-0.1350,-0.2376],[0.1764,-0.0854,0.0494,1,-0.4671,0.1135],[-0.4632,0.0193,-0.1350,-0.4671,1,-0.3656],[0.2939,0.2191,-0.2376,0.1135,-0.3656,1]]
generating_prec_matrix=np.linalg.inv(covar_generating_matrix)

In [90]:
rng=np.random.default_rng()
for i in range(d):
    for j in range(d):
        if (i!=j) and (rng.choice(2, p=[0.7, 0.3])==1):
            generating_prec_matrix[i,j]=0
            generating_prec_matrix[j,i]=0
generating_prec_matrix

array([[ 1.95468145,  0.        ,  0.        , -0.0045116 ,  0.7051812 ,
         0.        ],
       [ 0.        ,  1.33334865,  0.12669432,  0.08197025,  0.        ,
         0.        ],
       [ 0.        ,  0.12669432,  1.38388138,  0.        ,  0.        ,
         0.        ],
       [-0.0045116 ,  0.08197025,  0.        ,  1.29326295,  0.        ,
         0.        ],
       [ 0.7051812 ,  0.        ,  0.        ,  0.        ,  1.81765844,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.43282594]])

In [91]:
for i in range(d+1):
    if np.linalg.det(generating_prec_matrix[0:i, 0:i])<=0:
        print('not positive-definite')

In [92]:
covar_generating_matrix=np.linalg.inv(generating_prec_matrix)

In [93]:
data_sample=np.random.multivariate_normal(generating_mean,covar_generating_matrix, size=n)

Считаем ковариационную матрицу

In [94]:
d=data_sample.shape[1]
n=data_sample.shape[0]
sample_cov_matrix=np.cov(data_sample,rowvar=False)
sample_cov_matrix

array([[ 0.4695971 ,  0.00383157,  0.08398869, -0.13425562, -0.22975775,
         0.12761586],
       [ 0.00383157,  0.78679919,  0.07044771, -0.16703072, -0.07260823,
         0.03247373],
       [ 0.08398869,  0.07044771,  0.84327545, -0.3606347 , -0.08068782,
         0.06821478],
       [-0.13425562, -0.16703072, -0.3606347 ,  1.05924438,  0.08134802,
        -0.13008894],
       [-0.22975775, -0.07260823, -0.08068782,  0.08134802,  0.70708471,
        -0.0565959 ],
       [ 0.12761586,  0.03247373,  0.06821478, -0.13008894, -0.0565959 ,
         0.7151226 ]])

In [95]:
np.linalg.det(sample_cov_matrix)

0.10283412015166933

Cчитаем матрицу точности:

In [96]:
sample_prec_matrix=np.linalg.inv(sample_cov_matrix)
sample_prec_matrix

array([[ 2.71643172,  0.13452458, -0.07321247,  0.23053301,  0.83149895,
        -0.37613816],
       [ 0.13452458,  1.33088338, -0.01995091,  0.20436133,  0.15191965,
        -0.03333988],
       [-0.07321247, -0.01995091,  1.39904188,  0.45406097,  0.07912192,
        -0.03062145],
       [ 0.23053301,  0.20436133,  0.45406097,  1.17330101,  0.02244694,
         0.12148152],
       [ 0.83149895,  0.15191965,  0.07912192,  0.02244694,  1.70458032,
        -0.02384316],
       [-0.37613816, -0.03333988, -0.03062145,  0.12148152, -0.02384316,
         1.49013145]])

Считаем матрицу корреляций

In [97]:
sample_corr_matrix=np.corrcoef(data_sample, rowvar=False)

Выполняем алгоритм

In [98]:
graph_component = np.zeros(d) #это чтобы циклов не создавать -- будем соединять вершины только из разных компонент связности -- ноль это не компонента, а все вершины без ребер
graph=[]
for i in range (d):
    graph.append((i,i)) #граф, в который будем вершины добавлять -- добавляем(i,i), потому что далее нужно будет множество ребер графа и пар(i,i)
component_counter=1 #это чтобы знать, какой номер будет у новой компоненты связности, если мы создаем новую
found_pair=1 #счетчик, который проверяет, что мы нашли хотя бы одну вершину
#подбираем
epsilon=0.0000000001
approx_cov1_matrix=np.linalg.inv(np.diag(np.diag(np.linalg.inv(sample_cov_matrix)))) #это будет предыдущая по добавлению ребер ков. матрица, чтобы мы могли сравнить улучшение при добавлении каждого ребра
while found_pair!=0:
    found_pair=0
    corr_max=-1 #счетчик, который гарантирует, что мы выбираем ребро с наибольшим модулем корреляции
    for i in range (d):
        for j in range (d):
            if ((abs(sample_corr_matrix[i,j])>corr_max) and ((graph_component[i]!=graph_component[j]) or (graph_component[i]==0))) and (i!=j):#условия, чтобы не было циклов (включая из вершины в себя) -- ненулевые компоненты не могут быть одинаковые, и новая пара была лучбше по корреляции предыдущей
                found_pair=(i,j)
                corr_max=abs(sample_corr_matrix[i,j])
    if found_pair!=0:
        graph.append(found_pair)
        if (graph_component[found_pair[0]]==0) and (graph_component[found_pair[1]]==0):#новая компонента
            graph_component[found_pair[0]]=component_counter
            graph_component[found_pair[1]]=component_counter
            component_counter+=1
        elif (graph_component[found_pair[0]]==0): #присоединеяем к существующей компоненте
            graph_component[found_pair[0]]=graph_component[found_pair[1]]
        elif (graph_component[found_pair[1]]==0): #присоединеяем к существующей компоненте
            graph_component[found_pair[1]]=graph_component[found_pair[0]]
        else:
            abandonded_graph_component=graph_component[found_pair[0]] #переделываем номер компоненты под номер соединенной компоненты
            for i in range (d):
                if abandonded_graph_component==graph_component[i]:
                    graph_component[i]=graph_component[found_pair[1]]
        #это шаг алгоритма крускала -- ищем наибольшую корреляцию, не образующию циклов (не соединяющую элементы из одной компоненты связности), и добавляем её в дерево
        #теперь построим аппроксимацию ков. матрицы по дереву и сравним качество улучшения с предыдущей версией    
        #построим первую матрицу точности -- обратную к матрице ковариаций, с нулями в нужных местах, и запустим итерационное приближение из неё, чтобы найти матрицу, совпадающую на ребрах  
        #начальный пункт -- phi_i = approx_prec_matrix равен выборочному значению в графе и 0 вне графа, остальное строится от обратной матрицы к phi_i
        theta_a=[]#это цель итерационной процедуры -- найти матрицу, совпадающую (с учетом коэффициентов) с theta_a на элементах graph, и обратная которой равна 0 вне графа
        for i in range (len(graph)):
            if (graph[i][0]==graph[i][1]):
                theta_a.append(-1/2*sample_cov_matrix[graph[i][0],graph[i][1]])
            else:
                theta_a.append(-sample_cov_matrix[graph[i][0],graph[i][1]])
        theta_a=np.array(theta_a)
        phi_i_a=[]#часть phi_i, которая соответсвует элементам graph
        for i in range (len(graph)):
            phi_i_a.append(sample_prec_matrix[graph[i][0],graph[i][1]])
        phi_i_a=np.array(phi_i_a)
        approx_prec_matrix=np.zeros((d,d))
        for i in range (d):
            for j in range (d):
                if ((i,j) in graph) or((j,i) in graph):
                    approx_prec_matrix[i,j]=sample_prec_matrix[i,j]
        while np.linalg.det(approx_prec_matrix)<0:#"выпрямляем" матрицу, если её детерминант стал отрицательным, умножая её на 0.1 и складывая с собственной диагональю до умножения
            print('wrong sign')
            approx_prec_matrix=np.diag(np.diag(approx_prec_matrix))+(approx_prec_matrix/10)
            for i in range (d):
                phi_i_a[i]=1.1*phi_i_a[i]
            for i in range (len(graph)-d):
                phi_i_a[d+i]=0.1*phi_i_a[d+i]
        approx_cov_matrix=np.linalg.inv(approx_prec_matrix)
        while np.linalg.det(approx_cov_matrix)<epsilon:
            print('small determinant')
            approx_prec_matrix=approx_prec_matrix/2
            for i in range (d):
                phi_i_a[i]=phi_i_a[i]/2
            for i in range (len(graph)-d):
                phi_i_a[d+i]=phi_i_a[d+i]/2
            approx_cov_matrix=np.linalg.inv(approx_prec_matrix)
        theta_a_i=[]
        for i in range (len(graph)):
            if (graph[i][0]==graph[i][1]):
                theta_a_i.append(-1/2*approx_cov_matrix[graph[i][0],graph[i][1]])
            else:
                theta_a_i.append(-approx_cov_matrix[graph[i][0],graph[i][1]])
        theta_a_i=np.array(theta_a_i)
        delta=100
        epsilon=0.0000001
        while (delta>epsilon):#по сути градиентный поиск точки --здесь шаг такой, что при линейной функции мы бы сразу попали в искомую точку. Аллен это называет методом Ньютона для решения неявных уравнений
            #создаем матрицу Гамма_(1,1) -- составляем набор пар графа, считаем матрицу с ними, чтобы потом решить уравнение gamma_1_1_i*x=theta_a-theta_a_i и получить искомый сдвиг для phi_i_a
            gamma_1_1_i=np.zeros((len(graph), len(graph)))
            for i in range (len(graph)):
                for j in range (len(graph)):
                    coef_mult=1
                    if (graph[i][0]==graph[i][1]):#коэффиценты отвечающие -1/2 у theta -- два минуса всегда обращаются в плюс, так что минусы игнорируем во всех случаях
                        coef_mult/=2
                    if (graph[j][0]==graph[j][1]):
                        coef_mult/=2
                    gamma_1_1_i[i,j]=coef_mult*(approx_cov_matrix[graph[i][0],graph[j][1]]*approx_cov_matrix[graph[i][1],graph[j][0]]+approx_cov_matrix[graph[i][0],graph[j][0]]*approx_cov_matrix[graph[i][1],graph[j][1]]) #спариваем элементы из i-й и j-й пары 2 способами
            s=np.linalg.lstsq(gamma_1_1_i,theta_a-theta_a_i)[0] #предупреждение каждый раз вылезает, но оно вроде не важное
            alpha=0.001
            proper_step=False
            while (proper_step==False):#проверяем, что шаг не слишком большой: детерминант не стал отрицательным
                phi_i1_a=phi_i_a+alpha*s
                #пересчитываем все итерационные величины по phi_i1_a
                for i in range (len(graph)):
                    approx_prec_matrix[graph[i][0],graph[i][1]]=phi_i1_a[i]
                    approx_prec_matrix[graph[i][1],graph[i][0]]=phi_i1_a[i]
                approx_cov_matrix=np.linalg.inv(approx_prec_matrix)
                if np.linalg.det(approx_cov_matrix)>0:
                    proper_step=True
                else:
                    alpha/=10
            print(np.linalg.det(approx_cov_matrix))
            delta=np.sum(np.square(phi_i1_a-phi_i_a))#меряем размер шага в качестве критерия остановки
            phi_i_a=phi_i1_a
            for i in range (len(graph)):
                if (graph[i][0]==graph[i][1]):
                    theta_a_i[i]=-1/2*approx_cov_matrix[graph[i][0],graph[i][1]]
                else:
                    theta_a_i[i]=-approx_cov_matrix[graph[i][0],graph[i][1]]
        print(f'Детерминант старой матрицы -- {np.linalg.det(approx_cov1_matrix)}, новой -- {np.linalg.det(approx_cov_matrix)}')
        #проверяем, что добавление вершины эффективно
        delta_k=n*(math.log(np.linalg.det(approx_cov1_matrix)) - math.log(np.linalg.det(approx_cov_matrix))) + (n-1)*(np.sum(sample_cov_matrix*(np.linalg.inv(approx_cov1_matrix)-np.linalg.inv(approx_cov_matrix))))
        print(f'изменение параметра -- {delta_k}, функция распределения -- {sp.stats.chi2.cdf(delta_k, 1)}')
        print(f'cov1 -- \n{approx_cov1_matrix}\n cov -- \n{approx_cov_matrix}')
        #проверка на значимость
        if  (1-sp.stats.chi2.cdf(delta_k, 1) < 0.05/(d*(d+1)/2-(len(graph)-1))):
            approx_cov1_matrix=approx_cov_matrix
        else:
            print(f'prec -- \n{approx_prec_matrix}')
            print('stop!')
            graph.pop()
            break
if (found_pair==0):
    print('Закончились пары')

[[0.36813    0.         0.         0.         0.         0.        ]
 [0.         0.75138064 0.         0.         0.         0.        ]
 [0.         0.         0.71477489 0.         0.         0.        ]
 [0.         0.         0.         0.85229621 0.         0.        ]
 [0.         0.         0.         0.         0.58665467 0.        ]
 [0.         0.         0.         0.         0.         0.67108174]]
[[0.36813    0.         0.         0.         0.         0.        ]
 [0.         0.75138064 0.         0.         0.         0.        ]
 [0.         0.         0.71477489 0.         0.         0.        ]
 [0.         0.         0.         0.85229621 0.         0.        ]
 [0.         0.         0.         0.         0.58665467 0.        ]
 [0.         0.         0.         0.         0.         0.67108174]]
0.06640799975981948
0.0664754475030027
0.06654288185650573
0.06661030275800914
0.06667771014532375
0.06674510395639002
0.06681248412927848
0.0668798506021893
0.0669472033

C:\Users\arkad\AppData\Local\Temp\ipykernel_8360\696617263.py:63: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  s=np.linalg.lstsq(gamma_1_1_i,theta_a-theta_a_i)[0] #предупреждение каждый раз вылезает, но оно вроде не важное. Не уверен, что алгоритм работает, когда gamma_1_1_i вырождено, но использовал lstsq, чтобы он не ломался в этом случае


0.10091312361635846
0.10096660555737688
0.10102005647909222
0.1010734763747067
0.10112686523748503
0.10118022306075442
0.10123354983790483
0.1012868455623882
0.10134011022771905
0.10139334382747378
0.1014465463552908
0.10149971780487042
0.10155285816997472
0.10160596744442721
0.10165904562211313
0.10171209269697905
0.10176510866303261
0.10181809351434269
0.10187104724503926
0.10192396984931319
0.10197686132141595
0.10202972165565989
0.10208255084641764
0.10213534888812245
0.10218811577526786
0.1022408515024075
0.10229355606415504
0.10234622945518414
0.1023988716702283
0.10245148270408075
0.1025040625515941
0.10255661120768071
0.10260912866731209
0.10266161492551899
0.10271406997739134
0.102766493818078
0.10281888644278682
0.10287124784678425
0.10292357802539538
0.10297587697400393
0.10302814468805194
0.10308038116303982
0.103132586394526
0.10318476037812702
0.10323690310951747
0.10328901458442959
0.10334109479865337
0.10339314374803635
0.1034451614284837
0.10349714783595769
0.103549102

0.12163402529261991
0.12167376745984336
0.12171348019405616
0.12175316350609663
0.12179281740682371
0.12183244190711681
0.12187203701787575
0.12191160275002072
0.12195113911449222
0.12199064612225108
0.12203012378427815
0.1220695721115744
0.12210899111516098
0.12214838080607891
0.12218774119538911
0.1222270722941723
0.12226637411352914
0.12230564666457976
0.12234488995846414
0.12238410400634163
0.12242328881939137
0.12246244440881164
0.1225015707858204
0.12254066796165472
0.12257973594757086
0.12261877475484453
0.12265778439477035
0.12269676487866207
0.12273571621785238
0.12277463842369284
0.12281353150755425
0.12285239548082562
0.12289123035491507
0.12293003614124946
0.122968812851274
0.12300756049645253
0.12304627908826746
0.12308496863821962
0.12312362915782796
0.12316226065862992
0.12320086315218119
0.12323943665005553
0.12327798116384488
0.1233164967051591
0.1233549832856262
0.12339344091689185
0.12343186961061978
0.12347026937849144
0.12350864023220604
0.12354698218348029
0.12358

0.13766624366516525
0.1376932650359212
0.1377202636009741
0.13774723937519825
0.13777419237346789
0.13780112261065636
0.13782803010163686
0.13785491486128199
0.13788177690446335
0.13790861624605272
0.13793543290092064
0.13796222688393717
0.13798899820997193
0.13801574689389368
0.13804247295057046
0.13806917639486951
0.13809585724165757
0.13812251550580043
0.13814915120216312
0.13817576434560971
0.13820235495100378
0.13822892303320766
0.13825546860708302
0.13828199168749053
0.13830849228928985
0.1383349704273398
0.13836142611649832
0.1383878593716221
0.13841427020756678
0.13844065863918717
0.13846702468133712
0.13849336834886872
0.1385196896566339
0.13854598861948256
0.13857226525226402
0.13859851956982608
0.13862475158701554
0.13865096131867785
0.13867714877965734
0.1387033139847969
0.13872945694893807
0.13875557768692132
0.13878167621358564
0.13880775254376873
0.1388338066923067
0.13885983867403454
0.13888584850378544
0.1389118361963917
0.13893780176668358
0.1389637452294901
0.1389896

...

0.15649061665009076
0.15650074278010884
0.15651085930541767
0.15652096623459977
0.15653106357623114
0.15654115133888144
0.1565512295311134
0.15656129816148334
0.15657135723854093
0.1565814067708292
0.15659144676688502
0.15660147723523793
0.15661149818441153
0.15662150962292257
0.15663151155928123
0.15664150400199114
0.15665148695954956
0.15666146044044688
0.156671424453167
0.15668137900618753
0.15669132410797926
0.15670125976700636
0.15671118599172668
0.15672110279059145
0.1567310101720453
0.15674090814452638
0.15675079671646633
0.1567606758962902
0.15677054569241636
0.15678040611325694
0.1567902571672174
0.1568000988626967
0.15680993120808723
0.15681975421177488
0.15682956788213917
0.15683937222755287
0.15684916725638245
0.15685895297698763
0.15686872939772195
0.15687849652693225
0.15688825437295895
0.15689800294413578
0.15690774224879026
0.15691747229524328
0.15692719309180928
0.1569369046467962
0.1569466069685055
0.1569563000652321
0.1569659839452646
0.15697565861688506
0.1569853240

0.16106877345018764
0.16107450390863443
0.16108022879890482
0.1610859481262459
0.16109166189590043
0.1610973701131063
0.1611030727830969
0.1611087699111013
0.16111446150234332
0.16112014756204288
0.16112582809541476
0.16113150310766952
0.16113717260401292
0.16114283658964626
0.1611484950697661
0.16115414804956454
0.16115979553422904
0.16116543752894277
0.16117107403888373
0.16117670506922605
0.16118233062513876
0.16118795071178665
0.16119356533432963
0.16119917449792354
0.1612047782077193
0.16121037646886327
0.16121596928649753
0.16122155666575935
0.16122713861178176
0.16123271512969298
0.16123828622461675
0.1612438519016724
0.1612494121659747
0.16125496702263387
0.16126051647675566
0.1612660605334413
0.16127159919778739
0.1612771324748861
0.1612826603698252
0.16128818288768795
0.16129370003355295
0.16129921181249446
0.16130471822958214
0.1613102192898812
0.16131571499845251
0.1613212053603523
0.16132669038063227
0.1613321700643399
0.161337644416518
0.16134311344220492
0.16134857714643

0.16322633840559414
0.1632299609033477
0.16323357984260362
0.16323719522679334
0.1632408070593451
0.163244415343684
0.1632480200832318
0.16325162128140744
0.16325521894162612
0.16325881306730053
0.1632624036618398
0.16326599072864992
0.16326957427113378
0.16327315429269107
0.16327673079671853
0.16328030378660946
0.16328387326575422
0.16328743923754002
0.16329100170535074
0.16329456067256726
0.16329811614256726
0.16330166811872548
0.1633052166044132
0.16330876160299876
0.16331230311784747
0.16331584115232137
0.16331937570977945
0.16332290679357728
0.16332643440706804
0.1633299585536009
0.16333347923652258
0.16333699645917643
0.16334051022490267
0.16334402053703856
0.16334752739891817
0.16335103081387248
0.1633545307852293
0.1633580273163135
0.16336152041044674
0.16336501007094772
0.1633684963011319
0.16337197910431175
0.16337545848379664
0.1633789344428929
0.16338240698490375
0.16338587611312919
0.16338934183086648
0.1633928041414096
0.16339626304804938
0.16339971855407387
0.16340317066

0.07842929915079508
0.07847853773189489
0.07852774984653223
0.07857693548299047
0.07862609462962057
0.07867522727484096
0.0787243334071375
0.07877341301506312
0.07882246608723792
0.07887149261234899
0.0789204925791502
0.07896946597646208
0.079018412793172
0.0790673330182335
0.07911622664066673
0.07916509364955798
0.07921393403405945
0.07926274778338968
0.07931153488683282
0.07936029533373878
0.07940902911352324
0.07945773621566711
0.07950641662971694
0.07955507034528438
0.07960369735204618
0.07965229763974427
0.07970087119818528
0.07974941801724063
0.07979793808684646
0.07984643139700331
0.07989489793777622
0.0799433376992946
0.07999175067175164
0.080040136845405
0.08008849621057595
0.0801368287576497
0.08018513447707512
0.08023341335936446
0.08028166539509368
0.08032989057490185
0.0803780888894912
0.08042626032962716
0.08047440488613804
0.08052252254991499
0.08057061331191184
0.08061867716314501
0.08066671409469342
0.08071472409769835
0.08076270716336317
0.08081066328295353
0.08085859

C:\Users\arkad\AppData\Local\Temp\ipykernel_8360\696617263.py:174: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  s=np.linalg.lstsq(gamma_1_1_i,theta_a-theta_a_i)[0] #предупреждение каждый раз вылезает, но оно вроде не важное. Не уверен, что алгоритм работает, когда gamma_1_1_i вырождено, но использовал lstsq, чтобы он не ломался в этом случае


Получаем граф:

In [99]:
graph

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (0, 4), (3, 2)]